In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/Logistic_essay/asap-aes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import pandas as pd

df = pd.read_csv("./asap-aes/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1')
df = df.dropna(axis=1)
df = df.drop(columns=['rater1_domain1', 'rater2_domain1'])
df = df.drop(columns=['essay_id', 'essay_set'])
df.head()

,essay,domain1_score
0,"Dear local newspaper, I think effects computer...",8
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,"Dear @LOCATION1, I know having computers has a...",8


In [3]:
df_dev = pd.read_csv("./asap-aes/valid_set.tsv", sep='\t', encoding='ISO-8859-1')
df_dev = df_dev.dropna(axis=1)
df_dev = df_dev.drop(columns=['domain1_predictionid', 'essay_set'])
df_dev.head(), df_dev["essay"].shape

(   essay_id                                              essay
 0      1788  Dear @ORGANIZATION1, @CAPS1 more and more peop...
 1      1789  Dear @LOCATION1 Time @CAPS1 me tell you what I...
 2      1790  Dear Local newspaper, Have you been spending a...
 3      1791  Dear Readers, @CAPS1 you imagine how life woul...
 4      1792  Dear newspaper, I strongly believe that comput...,
 (4218,))

In [4]:
df_test = pd.read_csv("./asap-aes/test_set.tsv", sep='\t', encoding='ISO-8859-1')
df_test = df_dev.dropna(axis=1)
df_test.head()

,essay_id,essay
0,1788,"Dear @ORGANIZATION1, @CAPS1 more and more peop..."
1,1789,Dear @LOCATION1 Time @CAPS1 me tell you what I...
2,1790,"Dear Local newspaper, Have you been spending a..."
3,1791,"Dear Readers, @CAPS1 you imagine how life woul..."
4,1792,"Dear newspaper, I strongly believe that comput..."


In [5]:
df['essay'].shape, df_dev['essay'].shape, df_test['essay'].shape

((12976,), (4218,), (4218,))

In [6]:
df['domain1_score'].shape

(12976,)

In [7]:
df['essay'].shape

(12976,)

In [9]:
np.min(df["domain1_score"]), np.max(df["domain1_score"])

(0, 60)

In [11]:
import spacy 
nlp = spacy.load('en_core_web_md')
import numpy as np
from tqdm import tqdm
X, y, y0 = {}, {}, {}

In [36]:
X["train"] = np.array([nlp(df["essay"][i]).vector for i in tqdm(range(df['essay'].shape[0]))])

  1%|█▊                                                                                                                                                            | 152/12976 [00:09<11:03, 19.33it/s]

KeyboardInterrupt: 

In [13]:
y["train"] = np.array([df["domain1_score"][i] >=18 for i in tqdm(range(df['domain1_score'].shape[0]))])
np.sum(y["train"])  /12975

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12976/12976 [00:00<00:00, 155982.59it/s]


0.10057803468208093

In [6]:
X_text = np.array([df["essay"][i] for i in tqdm(range(df['essay'].shape[0]))])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12976/12976 [00:00<00:00, 103846.03it/s]


In [10]:
X_text[0], X_text.shape

("Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econo

In [ ]:
y["train"].shape, X["train"].shape

In [20]:
from sklearn.model_selection import train_test_split

X["train"], X["dev"], y["train"], y["dev"] = train_test_split(X["train"], y0["train"], test_size=0.1, random_state=42)

KeyError: 'train'

In [21]:
X_text_train, X_text_dev, y["train"], y["dev"] = train_test_split(X_text, y0["train"], test_size=0.1, random_state=42)

In [39]:
np.save("X_text_train.npy", X_text_train)
np.save("X_text_dev.npy", X_text_dev)

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X0 = {}
y0 = {}
X0["train"] = np.load("X_train.npy")
y0["train"] = np.load("y_train.npy")
X0["dev"] = np.load("X_dev.npy")
y0["dev"] = np.load("y_dev.npy")
np.all(X0["train"] == X["train"])

In [127]:
X["train"].shape, X["dev"].shape, y["train"].shape, np.sum(y["train"]), np.sum(y["dev"])

((11678, 300), (1298, 300), (11678,), 1181, 124)

In [128]:
np.save("X_train.npy", X["train"])
np.save("y_train.npy", y["train"])
np.save("X_dev.npy", X["dev"])
np.save("y_dev.npy", y["dev"])

### method1: label score > 9 to 1, lable score < 9 to 0, and use LR directly. all are Same as SST

### method2: treat data as 10 catagoried. Use some classification model, predict the score 1~10. Set 9 to be threshhold, see removing $k$ can let the lebel cross 9

# Save to new tsv file

In [52]:
df["essay"]

0        Dear local newspaper, I think effects computer...
1        Dear @CAPS1 @CAPS2, I believe that using compu...
2        Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...
3        Dear Local Newspaper, @CAPS1 I have found that...
4        Dear @LOCATION1, I know having computers has a...
                               ...                        
12971     In most stories mothers and daughters are eit...
12972     I never understood the meaning laughter is th...
12973    When you laugh, is @CAPS5 out of habit, or is ...
12974                                   Trippin' on fen...
12975     Many people believe that laughter can improve...
Name: essay, Length: 12976, dtype: object

In [61]:
X_array = []
for i in range(12976):
    X_array.append(df["essay"][i])

In [62]:
df["essay"].shape, len(X_array)

((12976,), 12976)

In [65]:
X_array = np.array(X_array)
X_array.shape

(12976,)

In [66]:
# split text file to dev and train
from sklearn.model_selection import train_test_split

df_train_X, df_dev_X, df_train_y, df_dev_y = train_test_split(X_array, y["train"], test_size=0.1, random_state=42)

In [67]:
df_train_X.shape, df_dev_X.shape

((11678,), (1298,))

In [68]:
np.all(df_dev_y == y["dev"])

True

In [71]:
df_train_X.shape

(11678,)

In [91]:
df_train_X[5279], df_train_y[5279]

('I had to be patience many times. I had to be patience when my @CAPS1 broke his ear-drum, I was waiting for x-rays, waiting to go on stage. These are sometimes when I had to be patience. When my @CAPS1 broke his ear-drum I was about five years old. I had to wait with my brother in the waiting room.  It felt like forever. It was winter, I was in my pjs, and so bored but worried about my @CAPS1. In @NUM1 grade I hit my wrist in a door and a wall. I was in the emergency room for @NUM2 to @NUM3 hours, but they were very good at there job. My mom and I were stuck watching @CAPS4 @CAPS5 on the tv they had. It was very cold in the waiting room plus I had @CAPS6 on my wrist. Finally, I was waiting to go on stage for my @NUM1 grade concert.  I was so nervous and the neeves kept growing. I waited for @NUM5 min but it felt like @NUM5 years. I was sitting next to my friend which helped a little. When we got on stage I song my heart out and was happy I waited. Those are sometimes which I had to be

In [93]:
df_train_X[5280], df_train_y[5280]

("The library is a public area where anyone can visit to find a book, movie, or magazine of their desire. Not everyone is interested in the same book genre. The library is meant to have almost any book someone could think of. If someone discovers a book or movie that offends them, they should not make a big deal about it. They should not be removed from the shelves of the library.      I am against the removal of offensive entertainment because it is not fair to everyone else that is a member of the library. What if a group of adults enjoyed a movie that someone else thought was disturbing? If the library took that movie off of the shelves, they would lose business due to one person. If a mother of a six year old boy wanted to report an inappropiate movie to the librarian, it would not be fair to anyone that is mature enough for that movie. It just is not fair to everyone else.     The library is public property. With that being said, it is the library's job to please the public's desi

In [94]:
df_train_X[5281], df_train_y[5281]

('The owners of the Empire State Building faced quite a few obstacles while attempting to allow dirigibles to dock at the mooring mast. One of the first obsticles they faced was having to modify and strengthen the steal frame.This was essential in order for the building to be able to support the mooring mast where the dirigibles were supposed to dock. Another great concern was safety, they feared an accident similar to the Hindenburg and thought of "how much worse the accident could have been (paragraph @NUM1) if it had occurred in a city like New York. The greatest obstacle they faced however was nature. The violent air currents caused the wind to constantly shift which would cause the docked aircrafts to "swivel around and around the mooring mast" (paragraph @NUM2). A law which prevented airships from flying too low also made this dream a great fear to overcome. This prohibited them from not only docking there, but even approaching the area. Although this dream may have seemed possib

In [90]:
for i in df_dev_y:
    if i not in [True, False]:
        print(i)

In [101]:
import csv
with open('./essay_dev.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')

    tsv_writer.writerow(['sentence', 'label'])
    for i in range(df_dev_X.shape[0]):
        tsv_writer.writerow([df_dev_X[i], int(df_dev_y[i])])

In [102]:
with open('./essay_train.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['sentence', 'label'])
    for i in range(df_train_X.shape[0]):
        tsv_writer.writerow([df_train_X[i], int(df_train_y[i])])

In [81]:
maxi_len = 0
for i in range(11678):
    len_i = len(df_train_X[i])
    if  len_i > maxi_len:
        maxi_len = len_i
        
for i in range(1298):
    len_i = len(df_dev_X[i])
    if  len_i > maxi_len:
        maxi_len = len_i

maxi_len

In [82]:
maxi_len

6098

In [84]:
with open("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_essay/data/sst2_dev.tsv") as file:
       
    # Passing the TSV file to 
    # reader() function
    # with tab delimiter
    # This function will
    # read data from file
    tsv_file = csv.reader(file, delimiter="\t")
     
    # printing data line by line
    for line in tsv_file:
        print(line[1])

label
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [87]:
with open("./essay_dev.tsv") as file:
       
    # Passing the TSV file to 
    # reader() function
    # with tab delimiter
    # This function will
    # read data from file
    tsv_file = csv.reader(file, delimiter="\t")
     
    # printing data line by line
    for line in tsv_file:
        print(line[1])

label
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
1
1
0
0
1
0
0
1
0
0
0
1
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
1
0
1
0
0
0
0
1
0
0
1
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
